In [6]:
from collections import defaultdict, deque

def solve_turning_red():
    # Input parsing
    l, b = map(int, input().split())
    initial_colors = input().strip()
    color_map = {'R': 0, 'G': 1, 'B': 2}
    light_colors = [color_map[c] for c in initial_colors]
    
    button_lights = defaultdict(list)
    light_buttons = defaultdict(list)
    
    for button_id in range(b):
        data = list(map(int, input().split()))
        k = data[0]
        lights = data[1:]
        button_lights[button_id] = lights
        for light in lights:
            light_buttons[light - 1].append(button_id)

    # Graph construction
    adjacency_list = defaultdict(list)
    for light, buttons in light_buttons.items():
        if len(buttons) == 2:
            adjacency_list[buttons[0]].append((buttons[1], light))
            adjacency_list[buttons[1]].append((buttons[0], light))
        elif len(buttons) == 1:
            adjacency_list[buttons[0]].append((None, light))

    # Process connected components
    visited = [False] * b
    result = 0

    def propagate(component, start_value):
        """Propagate values across the connected component."""
        queue = deque([(component[0], start_value)])
        values = {component[0]: start_value}
        total_presses = 0

        while queue:
            current_button, current_value = queue.popleft()
            for neighbor, light in adjacency_list[current_button]:
                if neighbor is None:
                    # Single-control light
                    if (current_value + light_colors[light]) % 3 != 0:
                        return float('inf')  # Inconsistent
                    continue
                required_value = (3 - light_colors[light] - current_value) % 3
                if neighbor in values:
                    if values[neighbor] != required_value:
                        return float('inf')  # Inconsistent
                else:
                    values[neighbor] = required_value
                    queue.append((neighbor, required_value))

        total_presses = sum(values.values())
        return total_presses

    for button in range(b):
        if not visited[button]:
            # Get all buttons in the connected component
            component = []
            queue = deque([button])
            while queue:
                btn = queue.popleft()
                if not visited[btn]:
                    visited[btn] = True
                    component.append(btn)
                    for neighbor, _ in adjacency_list[btn]:
                        if neighbor is not None and not visited[neighbor]:
                            queue.append(neighbor)

            # Try all 3 initial values and choose the best
            best_presses = float('inf')
            for start_value in range(3):
                presses = propagate(component, start_value)
                best_presses = min(best_presses, presses)

            if best_presses == float('inf'):
                print("impossible")
                return

            result += best_presses

    print(result)

# Run the solution
solve_turning_red()


 4 4
 GBRG
 2 1 2
 2 2 3
 2 3 4
 1 4


6
